# Project Gutenberg Analysis

Explain why and how

In [26]:
import numpy as np
import pandas as pd
import zipfile
import sys
import os
import pickle
import gzip
from datetime import datetime
from metainfo import readmetadata

In [2]:
DATA_ROOT = 'data'

In [3]:
time_start = datetime.now()
meta_data = readmetadata()
print('Read in ' + str(len(meta_data)) + ' books in ' + str(datetime.now() - time_start))
print(meta_data.shape)

Read in 56809 books in 0:05:58.948278
(56809, 8)


In [4]:
print(meta_data.columns)

Index(['id', 'author', 'title', 'downloads', 'LCC', 'subjects',
       'authoryearofbirth', 'authoryearofdeath'],
      dtype='object')


In [18]:
def get_book_text(book_id=None):
    if not book_id:
        print('Please pass in a book_id.')
        return
    
    file_path = os.path.join(DATA_ROOT, 'books', str(book_id) + '.zip')
    
    if not os.path.isfile(file_path):
#         print('File Does Not Exist')
        return
    
    with zipfile.ZipFile(file_path) as myzip:
        with myzip.open(str(book_id) + '.txt') as myfile:
            # This could be done in one line but split for readability
            # Plus, this only needs to be run once per file and the results are then saved
            try:
                raw_data = myfile.read().decode('utf-8')
                # This removes the Project Gutenberg Header
                book_text = ''.join(raw_data.split('***')[2:])
                # Removes new lines
                book_text = book_text.replace('\n', ' ').replace('\r', ' ')
                return book_text
            except:
                return None
    return None

In [24]:
def get_all_books_text():
    file_name = os.path.join(DATA_ROOT, 'books.text.pkl.gz')
    if os.path.isfile(file_name):
        return pickle.load(gzip.open(file_name, 'rb'))
    text = {}
    for book_id in meta_data['id']:
        book_text = get_book_text(book_id=book_id)
        if book_text:
            text[book_id] = book_text
    pickle.dump(text, gzip.open(file_name, 'wb'), protocol=-1)
    return text

In [28]:
time_start = datetime.now()
text = get_all_books_text()
print('loaded ' + str(len(text)) + ' books in ' + str(datetime.now() - time_start))

loaded 691 books in 0:00:01.428009
